# NHN AI EasyMaker SDK 사용 예제

이 노트북에서는 NHN AI EasyMaker SDK를 사용하여 하이퍼파라미터 튜닝 작업을 수행하는 방법을 보여줍니다.

In [1]:
# 필요한 라이브러리 임포트
import easymaker

In [ ]:
# 필요한 변수 설정
region="kr1"
appkey="YOUR_APP_KEY"

# 인증 토큰 발급 가이드 : https://docs.nhncloud.com/ko/nhncloud/ko/public-api/api-authentication/#_1
user_access_key_id="YOUR_ACCESS_KEY_ID"
secret_access_key="YOUR_SECRET_ACCESS_KEY"
access_token=easymaker.sample.get_access_token(user_access_key_id, secret_access_key),

# 튜닝 중 생성된 학습 결과를 업로드할 경로를 설정합니다.
# NHN Cloud Object Storage에 AI EasyMaker 시스템 계정 권한 추가 필요 : https://docs.nhncloud.com/ko/Machine%20Learning/AI%20EasyMaker/ko/console-guide/#1-nhn-cloud-object-storage-ai-easymaker
model_upload_uri = "obs://YOUR_OBS_URI"

## SDK 초기화

SDK를 사용하기 위해 초기화합니다.

In [ ]:
# SDK 초기화
easymaker.init(
    region=region,
    appkey=appkey,
    access_token=access_token,
)

## 실험 생성

새로운 실험을 생성합니다.

In [4]:
# 실험 생성
experiment = easymaker.Experiment().create(experiment_name="experiment_sample")
experiment.print_info()

[AI EasyMaker] Experiment 'experiment_sample' already exists. experiment_id: cskni154xcny
{
    "appKey": "qV5VIb2S1hr33zTS",
    "createdDatetime": "2025-05-28T13:44:46+09:00",
    "experimentId": "cskni154xcny",
    "experimentName": "experiment_sample",
    "experimentStatusCode": "ACTIVE",
    "tensorboardAccessUri": "https://alpha-kr1-proxy-easymaker.nhncloud.com:443/v1.0/appkeys/qV5VIb2S1hr33zTS/#/defaults/tensorboard/#scalars&regexInput=^(experiment_sample|cskni154xcny)/"
}


## 하이퍼파라미터 튜닝 생성

생성된 실험을 기반으로 하이퍼파라미터 튜닝을 생성합니다.

### 튜닝 생성에 필요한 정보 조회

In [5]:
# 인스턴스 목록 조회
instance_type_list = easymaker.HyperparameterTuning.get_instance_type_list()
for instance in instance_type_list:
    instance.print_info()
instance_type_name = instance_type_list[0].name

{
    "id": "35a73b57-58a7-434d-aa08-5249aaa95b3e",
    "name": "m2.c2m4",
    "vcpus": 2,
    "ram": 4096,
    "disk": 0,
    "gpu": false
}
{
    "id": "edc79d63-98c3-4b77-a2d4-482d70e6b554",
    "name": "m2.c4m8",
    "vcpus": 4,
    "ram": 8192,
    "disk": 0,
    "gpu": false
}
{
    "id": "eaddf5bd-2725-409a-ac57-044935747183",
    "name": "m2.c8m16",
    "vcpus": 8,
    "ram": 16384,
    "disk": 0,
    "gpu": false
}
{
    "id": "5b45e38f-8d54-4a8b-82e8-a104e13d464e",
    "name": "m2.c16m32",
    "vcpus": 16,
    "ram": 32768,
    "disk": 0,
    "gpu": false
}
{
    "id": "f5ebd3a7-f978-477d-8dcf-2a1578e29df3",
    "name": "c2.c2m2",
    "vcpus": 2,
    "ram": 2048,
    "disk": 0,
    "gpu": false
}
{
    "id": "57865308-7e1f-4931-aaea-1c36eb053900",
    "name": "c2.c4m4",
    "vcpus": 4,
    "ram": 4096,
    "disk": 0,
    "gpu": false
}
{
    "id": "c8f6212c-c259-414c-a6bd-118eb3bb6693",
    "name": "c2.c8m8",
    "vcpus": 8,
    "ram": 8192,
    "disk": 0,
    "gpu": false
}


In [ ]:
# 이미지 목록 조회
image_list = easymaker.HyperparameterTuning.get_image_list()
for image in image_list:
    image.print_info()
image_name = "Ubuntu 22.04 CPU TensorFlow Training"

In [ ]:
# 하이퍼파라미터 튜닝 생성
tuning = easymaker.HyperparameterTuning().run(
    experiment_id=experiment.experiment_id,
    hyperparameter_tuning_name="tuning_sample",
    image_name=image_name,
    instance_type_name="m2.c4m8",
    distributed_node_count=1,
    data_storage_size=300,
    source_dir_uri=easymaker.sample.classification_model.train.source_dir_uri,
    entry_point=easymaker.sample.classification_model.train.entry_point,
    model_upload_uri=model_upload_uri,
    hyperparameter_spec_list=[
        easymaker.HyperparameterSpec(
            hyperparameter_name="learning_rate",
            hyperparameter_type_code=easymaker.HYPERPARAMETER_TYPE_CODE.DOUBLE,
            hyperparameter_min_value="0.01",
            hyperparameter_max_value="0.05",
        ),
    ],
    metric_list=[
        easymaker.Metric(name="loss"),
        easymaker.Metric(name="accuracy"),
        easymaker.Metric(name="val_loss"),
        easymaker.Metric(name="val_accuracy"),
    ],
    metric_regex="([\w|-]+)\s*:\s*([+-]?\d*(\.\d+)?([Ee][+-]?\d+)?)",
    objective_metric_name="val_loss",
    objective_type_code=easymaker.OBJECTIVE_TYPE_CODE.MINIMIZE,
    parallel_trial_count=1,
    max_trial_count=10,
    tuning_strategy_name=easymaker.TUNING_STRATEGY.BAYESIAN_OPTIMIZATION,
    early_stopping_algorithm=easymaker.EARLY_STOPPING_ALGORITHM.MEDIAN,
    early_stopping_min_trial_count=3,
    early_stopping_start_step=4,
    timeout_hours=24,
    dataset_list=[
        easymaker.Dataset(
            dataset_name="train",
            data_uri=easymaker.sample.classification_model.train.dataset.train,
        ),
        easymaker.Dataset(
            dataset_name="test",
            data_uri=easymaker.sample.classification_model.train.dataset.test,
        ),
    ]
)
tuning.print_info()

## 하이퍼파라미터 튜닝 삭제

생성된 하이퍼파라미터 튜닝을 삭제합니다.

In [ ]:
# 하이퍼파라미터 튜닝 삭제
tuning.delete()